**Интроспекция** (*introspection*) – это возможность исследовать тип и свойства объектов во время выполнения (*at runtime*). Это позволяет писать код, в котором сама программа исследует свойства объектов, но также это полезно и для исследования классов и объектов "вручную".

В Python всё является объектом: числа, строки, функции, классы, модули. И у каждого объекта можно узнать, к какому классу (типу) он принадлежит, какие у него имеются поля и методы и как он устроен внутри. 

Python предлагает богатый инструментарий для интроспекции. Дополнительные инструменты можно найти в модуле [`inspect`](https://docs.python.org/3/library/inspect.html).

Для примера будем рассматривать класс `Cell`

In [ ]:
class Cell:
    """Базовый класс клетки"""
    domain = "Eukaryota"    # поле класса
    def __init__(self, cell_type, diameter_um):
        self.cell_type = cell_type          # тип клетки
        self.diameter_um = diameter_um      # диаметр в микрометрах
    
    def volume(self) -> float:
        """Приблизительный объём клетки (сферическая модель)."""
        r = self.diameter_um / 2
        return (4 / 3) * 3.14159 * r ** 3

## Базовые функции интроспекции



### `type()` и `id()`

Функция `type()` возвращает тип объекта – класс экземпляром которого является объект, переданный в аргумент функции.

In [ ]:
neuron1 = Cell("neuron", 25.0)
neuron2 = Cell("neuron", 20.0)

type(neuron1)

Функция `id()` возвращает уникальный идентификатор объекта. 

In [ ]:
id(neuron1), id(neuron2)

У разных объектов разные идентификаторы, и для таких объектов оператор `is` возвращает `False`:

In [ ]:
neuron1 is neuron2

### `isinstance()` и `issubclass()`

Функция `isinstance()` проверяет, является ли объект экземпляром класса. Причем функция вернет `True` и в том случае, если класс является предком класса, экземпляром которого является объект:

In [ ]:
neuron = Cell("neuron", 25.0)

isinstance(neuron, Cell), isinstance(neuron, object)

Функция `issubclass()` проверяет, является ли класс потомком другого класса:

In [ ]:
class Erythrocyte(Cell):
    pass

issubclass(Erythrocyte, Cell), issubclass(Erythrocyte, object)

### `callable()`

Функция `callable()` проверяет, является ли объект **вызываемым**. Вызываемыми являются:
- функции;
- методы;
- классы;
- объекты, для которых определен метод `__call__()`;
- асинхронные функции и корутины;
- лямбда выражения.

Декораторы и генераторные функции (c `yield`) также являются вызываемыми, так как представляют собой просто функции.

In [ ]:
x = 15

callable(x), callable(neuron.volume), callable(Cell)

In [ ]:
callable(lambda x: x ** 2)

## Специальные атрибуты

Специальные атрибуты (как и магические методы) начинаются и заканчиваются **двойным подчеркиванием** (*double underscore*), поэтому, называются сокращенно *dunder attributes*.

Вот некоторые специальные атрибуты классов:

In [ ]:
print(f"{ Cell.__name__ }")     # имя класса в виде строки
print(f"{ Cell.__module__ }")   # модуль, в котором определен класс
print(f"{ Cell.__doc__ }")      # документация
print(f"{ Cell.__bases__ }")    # родительские классы
print(f"{ Cell.__mro__ }")      # порядок разрешения методов

У объектов есть специальные атрибуты `__dict__` и `__class__`. Атрибут `__dict__` рассмотрен ниже. Атрибут `__class__` дает доступ к классу объекта:

In [ ]:
neuron1 = Cell("neuron", 25.0)
neuron2 = Cell("neuron", 2.0)

neuron1.__class__

In [ ]:
neuron2.__class__.__name__ 

Среди атрибутов функций интересны также `__name__`, `__doc__`, `__annotations__`, `__defaults__`, `__closure__`. Методы функций также обладают следующими интересными специальными атрибутами:

- `__self__` – возвращают объект, к которому привязан метод.
- `__func__` – исходная функции, которая не привязана к объекту.

In [ ]:
neuron1.volume.__self__

Обращение к методу через класс, также дает доступ к исходной функции:

In [ ]:
Cell.volume(neuron2)

In [ ]:
neuron1.volume.__func__ == Cell.volume

In [ ]:
neuron1.volume is neuron2.volume

In [ ]:
neuron1.volume.__func__ is neuron2.volume.__func__

## Поле `__dict__` и функция `vars()`

Поле `__dict__` объекта представляет собой словарь, содержащий данные, специфичные для экземпляра класса – простраство имен объекта, и называется **словарем атрибутов** (*attributes dictionary*). Важно иметь в виду, что `__dict__` не содержит методы объекта и унаследованные поля. Этот словарь можно менять напрямую, однако делать это не рекомендуется. Словарь `__dict__` отсутствует у объектов встроенных типов (`int`, `str`, `dict` и т.д.), а так же у объектов у которых определен `__slots__`.

In [ ]:
neuron = Cell("neuron", 25.0)
neuron.__dict__

Поле `__dict__` класса представляет собой объект класса `mappingproxy`, который содержит пространство имен класса, и в отличие от словаря, является неизменяемым. `__dict__` класса содержит данные, общие для всех экземпляров класса:
- методы объекта;
- методы класса;
- статические методы;
- специальные поля и методы.

Поля объектов не содержатся в `__dict__`. 

In [ ]:
Cell.__dict__

Встроенная функция-обертка `vars()` возвращает значение поля `__dict__` объекта:

In [ ]:
vars(neuron)

In [ ]:
vars(Cell)

## Метод `__dir__()` и функция `dir()`

Метод `__dir__()` возвращает список доступных имен для объекта. По-умолчанию сюда входят:
- поля объекта;
- поля класса;
- методы объекта;
- методы класса;
- статические методы;
- унаследованные поля и методы.

In [ ]:
dir(neuron)

Ко всем этим именам можно обращаться через `.`:

In [ ]:
neuron.__sizeof__()

Функция `dir()` также возвращает список имен, доступных для объекта. Фактически, `dir()` вызывает метод `__dir__()` у объекта и возвращает список, но уже отсортированныйв. Если метод `__dir__()` не определен у объекта, то функция `dir()` возвращает объединение атрибутов из `__dict__` объекта, класса и всех базовых классов (включая унаследованные). 

Часто имеет смысл отфильтровать имена, начинающиеся на `__`:

In [ ]:
[name for name in dir(neuron) if not name.startswith('__')]

### Функции `hasattr()`, `getattr()`, `setattr()`, `delattr()`

- `hasattr()` – проверка существования атрибута;
- `getattr()` – получение значения атрибута;
- `setattr()` – установка значения атрибута;
- `delattr()` – удаление атрибута.

Каждая функция имеет два параметра (`obj, name`). Первым аргументом принимает имя объекта, а вторым – имя атрибута (в виде строки). Функция `setattr()` имеет третий параметр (`value`) – значение, которое нужно установить. Функция `getattr()` имеет третий необязательный параметр (`default`). Если этот параметр задан (третьим аргументом), то если атрибут отсутствует у объекта, то функция вернет значение параметра.

Функция `hasattr()` вызвает `getattr()` и ловит `AttributeError`, если нет такого атрибута. 

Функция `getattr()` вызывает метод объекта `__getattribute__()` и если возникает исключение `AttributeError`, то вызывает метод объекта `__getattr__()`. 

Функция `setattr()` вызывает метод объекта `__setattr__()`.

Функция `delattr` вызывает метод объекта `__delattr__()`.

In [ ]:
neuron = Cell("neuron", 30.0)

hasattr(neuron, "diameter_um"), hasattr(neuron, "domain")

In [ ]:
getattr(neuron, "diameter_um")

In [ ]:
setattr(neuron, "diameter_um", 33.0)
getattr(neuron, "diameter_um")

In [ ]:
getattr(neuron, "diameter", False)